In [1]:
import  os

In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_dir: Path
    test_data_dir: Path
    model_name: str
    alpha: float
    l1_ratio: float
    max_iter: int
    tol: float
    target_column: str

In [6]:
from src.loan_payment_prediction.constants import*
from src.loan_payment_prediction.utils.common import read_yaml, create_directories

In [11]:
class ModelTrainerConfigManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.SGDRegressor
        schema = self.schema.TARGET_COLUMN

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_dir=config.train_data_dir,
            test_data_dir=config.test_data_dir,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            max_iter=params.max_iter,
            tol=params.tol,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import numpy as np
import os
from src.loan_payment_prediction import logger
from sklearn.linear_model import SGDRegressor
import joblib
from pathlib import Path


In [9]:



class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train_model(self):
        logger.info("Starting model training step")

        # Load data
        train_data = pd.read_csv(self.config.train_data_dir)
        test_data = pd.read_csv(self.config.test_data_dir)

        logger.info("Training and test data loaded")

        # Split features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        #  Memory optimization
        train_x = train_x.astype("float32")
        test_x = test_x.astype("float32")
        train_y = train_y.astype("float32")
        test_y = test_y.astype("float32")

        # Initialize SGDRegressor with ElasticNet penalty
        model = SGDRegressor(
            penalty="elasticnet",
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            max_iter=self.config.max_iter,
            tol=self.config.tol,
            random_state=42
        )

        logger.info("Training SGDRegressor (ElasticNet)")
        model.fit(train_x, train_y)

        # Save model
        os.makedirs(self.config.root_dir, exist_ok=True)
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(model, model_path)

        logger.info(f"Model saved at {model_path}")

        return model


In [12]:
try:
    logger.info("Starting model training pipeline")

    config = ModelTrainerConfigManager()
    model_trainer_config = config.get_model_trainer_config()

    logger.info("Model trainer configuration loaded successfully")

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()

    logger.info("Model training completed successfully")

except Exception as e:
    logger.exception("Error occurred during model training pipeline")
    raise
        

[2025-12-25 10:58:18,354]: INFO: Starting model training pipeline
[2025-12-25 10:58:18,364]: INFO: YAML file config\config.yaml loaded successfully.
[2025-12-25 10:58:18,373]: INFO: YAML file params.yaml loaded successfully.
[2025-12-25 10:58:18,378]: INFO: YAML file schema.yaml loaded successfully.
[2025-12-25 10:58:18,383]: INFO: Directory created at: artifacts
[2025-12-25 10:58:18,386]: INFO: Model trainer configuration loaded successfully
[2025-12-25 10:58:18,388]: INFO: Starting model training step
[2025-12-25 10:58:24,177]: INFO: Training and test data loaded
[2025-12-25 10:58:24,464]: INFO: Training SGDRegressor (ElasticNet)
[2025-12-25 10:58:27,250]: INFO: Model saved at artifacts/model_trainer\model.joblib
[2025-12-25 10:58:27,310]: INFO: Model training completed successfully
